In [42]:
%logstart -o -t

import sys

import numpy as np
#in case we need to repeat experiment
#np.random.seed(255)

import pandas as pd
pd.options.display.max_rows = 22

import matplotlib.pyplot as plt
plt.style.use('classic')

import seaborn as sns
sns.set()

#sklearn imports
from sklearn.cluster import KMeans
#from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV


#Use print instead of display when run as python script
pyscript = True

#Classifier verborsity where supported
verbose_level=1

#Sample Size (5963272)
#sampleN = 3000000
#SampleN = 5963000
sampleN = 100000

import time

#datafile
#datafile = 'NCDB_FULL_Removed_All_Missing_Values_Multi_Class.csv'
datafile = 'NCDB_FULL_Removed_All_Missing_Values_Binary_Class_Transformed.csv'

#Enable Optimization Algorithms
enable_grid_search = False
svm_c = 1
svm_gamma = 1
enable_rf_features = True
feature_all = False
defaultFeatures = ['P_AGE', 'V_YEAR', 'C_HOUR', 'C_YEAR', 'C_MNTH', 'C_CONF', 'C_WDAY', 'C_VEHS', 'P_USER', 'P_SEX']
enable_kmeans = True
enable_logistic_regression_l1 = True

# Enable Algorithms
enable_logistic_regression = True
enable_decision_tree = True
enable_svm = True
enable_knn = True
enable_mlp = True
enable_rf = True

#Multiclass classification, binary if falase
multiclass = False

C:\Users\pradeep\Anaconda3\lib\site-packages\IPython\core\magics\logging.py:130: UserWarning: Couldn't start log: Log file is already active: ipython_log.py
  warn("Couldn't start log: %s" % sys.exc_info()[1])


In [43]:
print("Sample size: {}".format(sampleN))

if multiclass:
    print("Multi-Class Classification: Enabled")
else:
    print("Multi-Class Classification: Disabled")

if enable_grid_search:
    print("Grid Search: Enabled")
else:
    print("Grid Search: Disabled")

if enable_rf_features:
    print("Feature Selection by RandomForest: Enabled")
else:
    print("Feature Selection by RandomForest: Disabled")
    print("Default Feature: {}".format(defaultFeatures))

if feature_all:
    print("All Features: Enabled")
else:
    print("All Features: Disabled")
    
if enable_kmeans:
    print("K-means: Enabled")
else:
    print("K-means: Disabled")

if enable_logistic_regression:
    print("Logistic Regression: Enabled")
else:
    print("Logistic Regression: Disabled")
    
if enable_decision_tree:
    print("Decision Tree: Enabled")
else:
    print("Decision Tree: Disabled")
    
if enable_svm:
    print("Support Vector Machines: Enabled")
else:
    print("Support Vector Machines: Disabled")

if  enable_knn:
    print("KNN: Enabled")
else:
    print("KNN: Disabled")
    
if enable_mlp:
    print("MLP: Enabled")
else:
    print("MLP: Disabled")
    
if enable_rf:
    print("Random Forest: Enabeld")
else:
    print("Random Forest: Disabled")

Sample size: 100000
Multi-Class Classification: Disabled
Grid Search: Disabled
Feature Selection by RandomForest: Enabled
All Features: Disabled
K-means: Enabled
Logistic Regression: Enabled
Decision Tree: Enabled
Support Vector Machines: Enabled
KNN: Enabled
MLP: Enabled
Random Forest: Enabeld


In [44]:
#sys.stdout=open("Initial_Binary_Model2.out", "w")

In [45]:
t_start =  time.time()
print(time.asctime( time.localtime(t_start) ))

Mon Nov 26 18:41:41 2018


In [46]:
#load data
df = pd.read_csv(datafile, engine = 'python')
print(df.head(2))

   C_YEAR  C_MNTH  C_WDAY  C_HOUR  C_VEHS  C_CONF  C_RCFG  C_WTHR  C_RSUR  \
0       1       1       1       1       2      34       2       1       1   
1       1       1       1       1       2      34       2       1       1   

   C_RALN  C_TRAF  V_YEAR  P_SEX  P_AGE  P_PSN  P_USER  P_ISEV  
0       1       1       2      0     33      1       1       2  
1       1       1       2      0     70      1       1       1  


In [47]:
print(df.isnull().sum().sum())

0


In [48]:
print(df[df.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())

0


In [49]:
df_cat = df.astype('category').copy()

In [50]:
totalRows = df_cat.index.size
print("Number of Rows in the dataset: {}".format(totalRows))

Number of Rows in the dataset: 3655334


In [51]:
print(df_cat.columns)

Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG',
       'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_YEAR', 'P_SEX', 'P_AGE',
       'P_PSN', 'P_USER', 'P_ISEV'],
      dtype='object')


In [52]:
#One-Hot-Encoding of categorical


## Convert Class Variable to Binary if multi class disabled

In [53]:
## Convert Class Variable to Binary
### Merge Injury and Fatality as a single class
### we will compare the results.
if multiclass:
    #Undersample the majority for the 3 class evaluation
    
    df_class = df_cat.copy()
    
    # subset fatal class
    is_fatal =  df_class['P_ISEV']==3
    is_fatal_count = is_fatal.sum()
    print("Number of Fatal: {}".format(is_fatal_count))
    df_class_fatal = df_class[is_fatal]
    print(df_class_fatal.head(2))
    
    # subset injury class
    is_injury =  df_class['P_ISEV']==2
    is_injury_count = is_injury.sum()
    print("Number of Injury: {}".format(is_injury_count))
    df_class_injury = df_class[is_injury]
    print(df_class_injury.head(2))
    
    # subset non_injury class
    is_safe =  df_class['P_ISEV']==1
    is_safe_count = is_safe.sum()
    print("Number of Non-Injury: {}".format(is_safe_count))
    df_class_safe = df_class[is_safe]
    print(df_class_safe.head(2))
    
    # get the size of fatal datafram
    min_size = df_class_fatal.index.size
    print("Size of Fatal Subset: {}".format(min_size))
    
    # get size of injury
    print("Size of injury Subset: {}".format(df_class_injury.index.size))
    
    # size of non-fatal
    print("Size of non-fatal Subset: {}".format(df_class_safe.index.size))
    
    # randomly sample n number of injury and no injury and append to fatal
    df_class_injury_select = df_class_injury.sample(n=min_size)
    print("Shape of injury sampled dataframe: {}".format(df_class_injury_select.shape))
    df_class_safe_select = df_class_safe.sample(n=min_size)
    print("Shape of nom-injury sampled dataframe: {}".format(df_class_safe_select.shape))
    
    #concat the three dataframes
    df_underSample = pd.concat([df_class_fatal, df_class_injury_select, df_class_safe_select])
    print(df_underSample.shape)
    
    #TBD
    if sampleN < df_underSample.index.size:
        df_sample = df_underSample.sample(n=sampleN)
    else:
        df_sample = df_underSample.sample(n=df_underSample.index.size)
    
    #perform the conversion in two steps to avoid any unwanted side effects
    df_sample['P_ISEV'] = df_sample['P_ISEV'].map({1: 'safe', 2: 'injury', 3:'fatal'})
    df_sample['P_ISEV'] = df_sample['P_ISEV'].map({'safe': '0', 'injury': '1', 'fatal':'2'})
    print((df_sample['P_ISEV']=='0').sum())
    print((df_sample['P_ISEV']=='1').sum())
    print((df_sample['P_ISEV']=='2').sum())
    print(df_sample['P_ISEV'].unique())
else:
    df_class = df_cat.copy()

    #perform the conversion in two steps to avoid any unwanted side effects
    df_class['P_ISEV'] = df_class['P_ISEV'].map({1: 'safe', 2: 'injury', 3:'fatal'})
    df_class['P_ISEV'] = df_class['P_ISEV'].map({'safe': '0', 'injury': '1', 'fatal':'1'})
    print((df_class['P_ISEV']=='0').sum())
    print((df_class['P_ISEV']=='1').sum())
    print(df_class['P_ISEV'].unique())
    
    df_sample = df_class.sample(n=sampleN)

print("Size of dataframe for modeling: {}".format(df_sample.index.size))

1570775
2084559
['1' '0']
Size of dataframe for modeling: 100000


In [54]:
print(df_sample[df_sample.columns].apply(lambda x: x.astype(str).str.contains('[^0-9]')).sum().sum())

0


## Split Training and Testing for Binary class

In [55]:
#Split between data and class
Y = df_sample[df_sample.columns[-1]]
X = df_sample[df_sample.columns[0:df_sample.columns.size -1]]

#### Split Test(70%) and Train (30%) for Bianry class 

In [56]:
#sprint into train and test 70/30
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [57]:
#print(Xbinary_train, Xbinary_test, Ybinary_train, Ybinary_test)

## Write cleaned data to file for future use.

In [58]:
#lets write the datafile for future use
df_sample.to_csv('cleansimple_binary.csv', encoding='utf-8', index=False)

## Clustering based on K-Means Clustering

In [59]:
if enable_kmeans:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    print("K-Means Clustering: Start")
    kmeans = KMeans(n_clusters=3, init='random', n_init=10, max_iter=100, tol=1e-04, verbose= verbose_level)
    print("K-Means Clustering: Build")
    ykm = kmeans.fit(X)
    
    if pyscript:
        print(ykm.cluster_centers_)
        print(ykm.labels_)
    else:
        display(ykm.cluster_centers_)
        display(ykm.labels_)
    
    print("K-Means Clustering: End")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))


Mon Nov 26 18:43:23 2018
K-Means Clustering: Start
K-Means Clustering: Build
Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 26561969.787934925
start iteration
done sorting
end inner loop
Iteration 1, inertia 23492709.713582315
start iteration
done sorting
end inner loop
Iteration 2, inertia 23233139.61317361
start iteration
done sorting
end inner loop
Iteration 3, inertia 23177362.1348935
start iteration
done sorting
end inner loop
Iteration 4, inertia 23156903.29204248
start iteration
done sorting
end inner loop
Iteration 5, inertia 23147522.31727532
start iteration
done sorting
end inner loop
Iteration 6, inertia 23145392.731283978
start iteration
done sorting
end inner loop
Iteration 7, inertia 23144121.794038687
start iteration
done sorting
end inner loop
Iteration 8, inertia 23143484.01134267
start iteration
done sorting
end inner loop
Iteration 9, inertia 23143188.60329497
start iteration
done sorting
end inner loop
Iteration 10, inertia 

end inner loop
Iteration 2, inertia 23437661.496974103
start iteration
done sorting
end inner loop
Iteration 3, inertia 23324480.17750532
start iteration
done sorting
end inner loop
Iteration 4, inertia 23226627.80190758
start iteration
done sorting
end inner loop
Iteration 5, inertia 23165500.833236557
start iteration
done sorting
end inner loop
Iteration 6, inertia 23122967.577533226
start iteration
done sorting
end inner loop
Iteration 7, inertia 23101004.036650516
start iteration
done sorting
end inner loop
Iteration 8, inertia 23082805.640711382
start iteration
done sorting
end inner loop
Iteration 9, inertia 23073480.322553482
start iteration
done sorting
end inner loop
Iteration 10, inertia 23069791.8877839
start iteration
done sorting
end inner loop
Iteration 11, inertia 23065712.3764644
start iteration
done sorting
end inner loop
Iteration 12, inertia 23061559.610930458
start iteration
done sorting
end inner loop
Iteration 13, inertia 23060580.234571684
start iteration
done so

## Feature selection using Random Forest

### Feature Selection 

### Get the inportant features from random forest

In [60]:
if enable_rf_features:
    print("Random Forest Feature Selection: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    forest = RandomForestClassifier(n_estimators=250, random_state=0, n_jobs=-1, verbose=verbose_level)
    print("Random Forest Feature Selection: Fit Start")
    forest.fit(X, Y)
    print("Random Forest Feature Selection: Fit")

    importFeatures = forest.feature_importances_
    print("Random Forest Feature Selection: Feature Importance")
    print(importFeatures)
    
    indices = np.argsort(importFeatures)[::-1]
    print(indices)
    featureLabel = X.columns[0:]
    print(featureLabel)
    rankedFeature = []
    for f in range(X.shape[1]):
        rankedFeature.append(featureLabel[indices[f]])
        print("%2d) %-*s %f" % (f+1, 30,  featureLabel[indices[f]], importFeatures[indices[f]]))
    print(rankedFeature)

Random Forest Feature Selection: Start
Mon Nov 26 18:43:28 2018
Random Forest Feature Selection: Fit Start


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    4.7s


Random Forest Feature Selection: Fit
Random Forest Feature Selection: Feature Importance
[0.13448741 0.12047212 0.09385607 0.06884522 0.03104145 0.08022681
 0.02907444 0.0337799  0.03084771 0.02953504 0.03820262 0.0425893
 0.02938713 0.19560652 0.01221676 0.0298315 ]
[13  0  1  2  5  3 11 10  7  4  8 15  9 12  6 14]
Index(['C_YEAR', 'C_MNTH', 'C_WDAY', 'C_HOUR', 'C_VEHS', 'C_CONF', 'C_RCFG',
       'C_WTHR', 'C_RSUR', 'C_RALN', 'C_TRAF', 'V_YEAR', 'P_SEX', 'P_AGE',
       'P_PSN', 'P_USER'],
      dtype='object')
 1) P_AGE                          0.195607
 2) C_YEAR                         0.134487
 3) C_MNTH                         0.120472
 4) C_WDAY                         0.093856
 5) C_CONF                         0.080227
 6) C_HOUR                         0.068845
 7) V_YEAR                         0.042589
 8) C_TRAF                         0.038203
 9) C_WTHR                         0.033780
10) C_VEHS                         0.031041
11) C_RSUR                         0.0308

[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    6.5s finished


### Reduce the number of features

In [61]:
#select features that contribute more than 0.05
if enable_rf_features:
    X_Selected = X[rankedFeature[0:10]]
else:
    if feature_all:
        X_Selected = X
    else:
        X_Selected = X[defaultFeatures]

if pyscript:
    print(X_Selected)
else:
    display(X_Selected)

t_end =  time.time()
print(time.asctime( time.localtime(t_end) ))

print("Random Forest Feature Selection: End")


        P_AGE C_YEAR C_MNTH C_WDAY C_CONF C_HOUR V_YEAR C_TRAF C_WTHR C_VEHS
1497736     9      7      7      5     21      3      2      2      1      2
3059749    28     15      8      5     36      1      4      7      1      2
1908179    19      9      6      6     31      4      2      1      1      3
737129     15      4      3      4     31      3      1      1      1      2
345811     32      2      7      4     31      2      3      2      1      2
3372130    23     17      6      2     33      2      4      1      1      3
2647942    19     13      5      5     35      3      3      7      1      2
1760000    12      8     10      1      6      4      3      7      4      1
1321151    23      6      9      6     31      2      3      7      2      2
942120     53      5      1      4     21      2      3      7      1      3
3425156    68     17      9      5     35      1      2      1      1      2
...       ...    ...    ...    ...    ...    ...    ...    ...    ...    ...

In [62]:
print("Split Test and Train based on Selected Features")
#sprint into train and test 70/30
X_train, X_test, Y_train, Y_test = train_test_split(X_Selected, Y, test_size=0.3, random_state=0)

Split Test and Train based on Selected Features


### SVM GridSearch for Optimal Parms

In [63]:
#This operation is computationaly expensive.
if enable_grid_search:
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    param_grid = {'C':[0.1, 1, 10, 100, 1000], 'gamma':[1, 0.1, 0.01, 0.001, 0.0001]}
    grid = GridSearchCV(SVC(), param_grid, verbose=verbose_level)
    grid.fit(X_train, Y_train)
    print(grid.best_params_)
    svm_c = grid.best_params_.get('C')
    svm_gamma = grid.best_params_.get('gamma')
    print(grid.best_estimator_)
    grid_predictions = grid.predict(X_test)
    cfn_matrix_grid = confusion_matrix(Y_test, grid_predictions)
    print(cfn_matrix_grid)
    print(classification_report(Y_test,grid_predictions))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

## Logistic Regression Model

In [64]:
if enable_logistic_regression:
    print("Logistic Regression: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(C=1, random_state=0, verbose=verbose_level)
    print("Logistic Regression: Fit")
    lr.fit(X_train, Y_train)
    print("Logistic Regression: Predict")
    y_pred = lr.predict(X_test)

    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    # print the intercept (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Intercept")
    print(lr.intercept_)

    # print the coeficients (Note: one vs rest => 1 vs 2and3, 2 vs 1and3, 3 vs 1and2)
    print("Logistic Regression: Coefficients")
    print(lr.coef_)

    print("Logistic Regression: Confusion Matrix")
    cnf_matrix_lg = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg)
    
    print("Logistic Regression: Classification Report")
    print(classification_report(Y_test, y_pred))

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Logistic Regression: End")

Logistic Regression: Start
Mon Nov 26 18:43:35 2018
Logistic Regression: Fit
[LibLinear]Logistic Regression: Predict
Accuracy of logistic regression classifier on train set: 0.59
Accuracy of logistic regression classifier on test set: 0.59
Logistic Regression: Intercept
[1.06775746]
Logistic Regression: Coefficients
[[ 0.00638004  0.00118896  0.00125368 -0.00118633 -0.00154397 -0.06940412
  -0.09144479  0.05169274  0.05279483 -0.43455419]]
Logistic Regression: Confusion Matrix
[[ 3433  9330]
 [ 2941 14296]]
Logistic Regression: Classification Report
             precision    recall  f1-score   support

          0       0.54      0.27      0.36     12763
          1       0.61      0.83      0.70     17237

avg / total       0.58      0.59      0.55     30000

Mon Nov 26 18:43:36 2018
Logistic Regression: End


### Logistic Regression with L1 Regularization

In [65]:
if enable_logistic_regression_l1:
    # with L1 regularization
    print("Logistic Regression with L1 Regularization: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    lr = LogisticRegression(penalty='l1', C=1, random_state=0, verbose=verbose_level)
    print("Logistic Regression with L1 Regularization: Fit")
    lr.fit(X_train, Y_train)
    
    print("Logistic Regression with L1 Regularization: Predict")
    y_pred = lr.predict(X_test)
    
    print('Accuracy of logistic regression classifier on train set: {:.2f}'.format(lr.score(X_train, Y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

    print("Logistic Regression with L1 Regularization: Confusion Matrix")
    cnf_matrix_lg_l1 = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_lg_l1)
    
    print(classification_report(Y_test,y_pred))
    print("Logistic Regression with L1 Regularization: Classification Report")

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Logistic Regression with L1 Regularization: End")

Logistic Regression with L1 Regularization: Start
Mon Nov 26 18:43:36 2018
Logistic Regression with L1 Regularization: Fit
[LibLinear]Logistic Regression with L1 Regularization: Predict
Accuracy of logistic regression classifier on train set: 0.59
Accuracy of logistic regression classifier on test set: 0.59
Logistic Regression with L1 Regularization: Confusion Matrix
[[ 3436  9327]
 [ 2941 14296]]
             precision    recall  f1-score   support

          0       0.54      0.27      0.36     12763
          1       0.61      0.83      0.70     17237

avg / total       0.58      0.59      0.55     30000

Logistic Regression with L1 Regularization: Classification Report
Mon Nov 26 18:43:37 2018
Logistic Regression with L1 Regularization: End


### Decision Tree

In [66]:
if enable_decision_tree:
    print("Decision Tree: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    tree = DecisionTreeClassifier(criterion='entropy',max_depth=50, random_state=0)
    print("Decision Tree: Fit")
    tree.fit(X_train, Y_train)
    
    print("Decision Tree: Predict")
    y_pred = tree.predict(X_test)
    print('Accuracy of Decision Tree classifier on train set: {:.2f}'.format(tree.score(X_train, Y_train)))
    print('Accuracy of Decision Tree classifier on test set: {:.2f}'.format(tree.score(X_test, Y_test)))
    
    cnf_matrix_dt = confusion_matrix(Y_test, y_pred)
    print("Decision Tree: Confusion Matrix")
    print(cnf_matrix_dt)
    print("Decision Tree: Classification Report")
    print(classification_report(Y_test,y_pred))
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    print("Decision Tree: End")

Decision Tree: Start
Mon Nov 26 18:43:37 2018
Decision Tree: Fit
Decision Tree: Predict
Accuracy of Decision Tree classifier on train set: 1.00
Accuracy of Decision Tree classifier on test set: 0.55
Decision Tree: Confusion Matrix
[[ 6177  6586]
 [ 6847 10390]]
Decision Tree: Classification Report
             precision    recall  f1-score   support

          0       0.47      0.48      0.48     12763
          1       0.61      0.60      0.61     17237

avg / total       0.55      0.55      0.55     30000

Mon Nov 26 18:43:38 2018
Decision Tree: End


### K-N-N

In [67]:
if enable_knn:
    print("KNN: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
    print("KNN: Fit")
    knn.fit(X_train, Y_train)
    
    print("KNN: Predict")
    y_pred = knn.predict(X_test)
    print('Accuracy of KNN classifier on train set: {:.2f}'.format(knn.score(X_train, Y_train)))
    print('Accuracy of KNN classifier on test set: {:.2f}'.format(knn.score(X_test, Y_test)))
    
    print("KNN: Confusion Matrix")
    cnf_matrix_knn = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_knn)

    print("KNN: Classification Report")
    print(classification_report(Y_test,y_pred))

    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))

    print("KNN: End")

KNN: Start
Mon Nov 26 18:43:38 2018
KNN: Fit
KNN: Predict
Accuracy of KNN classifier on train set: 0.72
Accuracy of KNN classifier on test set: 0.56
KNN: Confusion Matrix
[[ 5603  7160]
 [ 5907 11330]]
KNN: Classification Report
             precision    recall  f1-score   support

          0       0.49      0.44      0.46     12763
          1       0.61      0.66      0.63     17237

avg / total       0.56      0.56      0.56     30000

Mon Nov 26 18:43:44 2018
KNN: End


### SVM

In [68]:
if enable_svm:
    print("SVM: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    #svm = SVC(C=1, random_state=0, kernel='sigmoid', verbose=True)
    #svm = SVC(C=1, random_state=0, kernel='linear', verbose=True, cache_size=200)
    #svm = SVC(C=svm_c, gamma=svm_gamma, verbose = verbose_level)
    svm = SVC(C=1, verbose = verbose_level)
    print("SVM: Fit")
    svm.fit(X_train, Y_train)
    
    print("SVM: Predict")
    y_pred = svm.predict(X_test)
    
    print('Accuracy of SVM classifier on train set: {:.2f}'.format(svm.score(X_train, Y_train)))
    print('Accuracy of SVM classifier on test set: {:.2f}'.format(svm.score(X_test, Y_test)))
    
    print("SVM: Confusion Matrix")
    cnf_matrix_svm = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_svm)
    
    print("SVM: Classfication Report")
    print(classification_report(Y_test,y_pred))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("SVM: End")

SVM: Start
Mon Nov 26 18:43:44 2018
SVM: Fit
[LibSVM]SVM: Predict
Accuracy of SVM classifier on train set: 0.80
Accuracy of SVM classifier on test set: 0.59
SVM: Confusion Matrix
[[ 4887  7876]
 [ 4542 12695]]
SVM: Classfication Report
             precision    recall  f1-score   support

          0       0.52      0.38      0.44     12763
          1       0.62      0.74      0.67     17237

avg / total       0.58      0.59      0.57     30000

Mon Nov 26 18:51:28 2018
SVM: End


## ANN - Multilayer Perceptron

In [69]:
if enable_mlp:
    print("Multilayer Preceptron: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    
    #mlpc = MLPClassifier(alpha=1)
    mlpc = MLPClassifier(hidden_layer_sizes=(12, 12, 12), max_iter=100, verbose=verbose_level)
    
    #mlp = multilayer_perceptron(n_hidden =2, activation='logistic', algorithm='sgd', random_state=3)
    print("Multilayer Preceptron: fit")
    mlpc.fit(X_train, Y_train)
    
    print("Multilayer Preceptron: Predict")
    y_pred = mlpc.predict(X_test)

    print('Accuracy of Multilayer Perceptron classifier on train set: {:.2f}'.format(mlpc.score(X_train, Y_train)))
    print('Accuracy of Multilayer Perceptron classifier on test set: {:.2f}'.format(mlpc.score(X_test, Y_test)))
    
    print("Multilayer Preceptron: Confusion Matrix")
    cnf_matrix_mlp = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_mlp)
    
    print("Multilayer Preceptron: Classificiation Report")
    print(classification_report(Y_test,y_pred))
    
    t_end =  time.time()
    print(time.asctime( time.localtime(t_end) ))
    
    print("Multilayer Preceptron: End")

Multilayer Preceptron: Start
Mon Nov 26 18:51:28 2018
Multilayer Preceptron: fit
Iteration 1, loss = 0.67914602
Iteration 2, loss = 0.66219475
Iteration 3, loss = 0.65891572
Iteration 4, loss = 0.65661332
Iteration 5, loss = 0.65567053
Iteration 6, loss = 0.65428996
Iteration 7, loss = 0.65297567
Iteration 8, loss = 0.65231010
Iteration 9, loss = 0.65208932
Iteration 10, loss = 0.65218485
Iteration 11, loss = 0.65101333
Iteration 12, loss = 0.65133399
Iteration 13, loss = 0.65122149
Iteration 14, loss = 0.65063030
Iteration 15, loss = 0.65050850
Iteration 16, loss = 0.65098891
Iteration 17, loss = 0.65024892
Iteration 18, loss = 0.65023059
Iteration 19, loss = 0.64995565
Iteration 20, loss = 0.65024483
Iteration 21, loss = 0.65015239
Iteration 22, loss = 0.64962320
Iteration 23, loss = 0.64957040
Iteration 24, loss = 0.65032973
Iteration 25, loss = 0.64940938
Iteration 26, loss = 0.64952597
Iteration 27, loss = 0.64922470
Iteration 28, loss = 0.64942012
Iteration 29, loss = 0.64945533


### Random forest

In [70]:
if enable_rf:
    print("Ensemble (Bagging): Random Forest: Start")
    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    forest = RandomForestClassifier(criterion='entropy', n_estimators=50, random_state=0, n_jobs=2, verbose=verbose_level)
    print("Ensemble (Bagging): Random Forest: Fit")
    forest.fit(X_train, Y_train)
    
    print("Ensemble (Bagging): Random Forest: Predict")
    y_pred = forest.predict(X_test)
    
    print('Accuracy of RandomForest classifier on train set: {:.2f}'.format(forest.score(X_train, Y_train)))
    print('Accuracy of RandomForest classifier on test set: {:.2f}'.format(forest.score(X_test, Y_test)))
    
    print("Ensemble (Bagging): Random Forest: Confusion Matrix")
    cnf_matrix_rf = confusion_matrix(Y_test, y_pred)
    print(cnf_matrix_rf)
    
    print("Ensemble (Bagging): Random Forest: Classification Report")
    print(classification_report(Y_test,y_pred))

    t_start =  time.time()
    print(time.asctime( time.localtime(t_start) ))
    
    print("Ensemble (Bagging): Random Forest: End")

Ensemble (Bagging): Random Forest: Start
Mon Nov 26 18:51:33 2018
Ensemble (Bagging): Random Forest: Fit


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    1.6s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    1.7s finished


Ensemble (Bagging): Random Forest: Predict


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.3s finished


Accuracy of RandomForest classifier on train set: 1.00


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.1s finished


Accuracy of RandomForest classifier on test set: 0.59
Ensemble (Bagging): Random Forest: Confusion Matrix
[[ 5831  6932]
 [ 5353 11884]]
Ensemble (Bagging): Random Forest: Classification Report
             precision    recall  f1-score   support

          0       0.52      0.46      0.49     12763
          1       0.63      0.69      0.66     17237

avg / total       0.58      0.59      0.59     30000

Mon Nov 26 18:51:36 2018
Ensemble (Bagging): Random Forest: End


In [71]:

#check sigmoid and rbf
#from sklearn.ensemble import BaggingClassifier
#from sklearn.svm import SVC
#clf = BaggingClassifier(SVC(C=1.0,
#        cache_size=200,
#        class_weight=None,
#        coef0=0.0,
#        decision_function_shape=None,
#        degree=3,
#        gamma='auto',
#        kernel='linear',
#        max_iter=-1,
#        probability=False,
#        random_state=None,
#        shrinking=True,
#        tol=0.001,
#        verbose=False,
#        ))

In [72]:
#clf.fit(X_train, Y_train)

In [73]:
t_end =  time.time()
print(time.asctime( time.localtime(t_end) ))

Mon Nov 26 18:51:36 2018


In [74]:
#sys.stdout.close()
%logstop